Done by :

    Rayhane Talbi
    
                                 *** Spam Filtering ***
 
                                                                                                                    PYTHON 3

LIBRAIRIES

In [1]:
import pandas as pd 

In [2]:
import numpy as np

In [3]:
import re

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords

In [6]:
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.stem import WordNetLemmatizer


In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
from collections import Counter

DATA VISUALISATION

In [10]:
df = pd.read_csv('SMSSpamCollection', header=None, sep='\t', names=['y', 'x'])

In [11]:
df.head()

,y,x
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
df.groupby('y').count() #Counting occurencies of hams and spams

,x
y,
ham,4825
spam,747


DATA PREPROCESSING

In [13]:
def clear_data(df_):
    df_ = df_.replace(to_replace = '', value = np.NaN) 

    #Delete line with nan (to be sure)
    df1 = df_.dropna(how='any')

    #Supprimer les colonnes n'ayant qu'une valeur unique
    #df1 = df1.loc[:, (df1 != df1.iloc[0]).any()] 

    #Delete column with only distinct value
    """
    for col in df1.columns:
        if(pd.Series(df1[col]).dtype == "object"):
            if len(df1[col].unique()) == len(df1):
                df1 = df1.drop(col, axis=1) 
    """
    #Delete lines in double
    df2=df1.drop_duplicates(keep='first')
    df2 = df2.reset_index()
    return df2

In [14]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [15]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'(0\w{3} \w{3} \w{4})|(0\w{10})|(0\w{3} \w{3} \w{2} \w{2})','phonenumbr', text)
    text = text.replace(r'£|\$', 'moneysymb')  
    text = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', text)
    text = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr',text)
    
    text = re.sub(r'\d+','',text)
    text = (re.compile('[^a-z ]')).sub('',text)
    text = text.strip()
    tokens = nltk.word_tokenize(text)
    result = [token for token in tokens if not token in stop_words]
    result = [lemmatizer.lemmatize(word) for word in result]
    return result

In [16]:
#Shuffle data
def shuffle_data(df):
    return df.sample(frac=1)

In [17]:
cdf = df.copy()
cdf['x'] = cdf['x'].apply(lambda i: preprocess(i))

SPLIT DATA

In [18]:
sf = shuffle_data(cdf)
sf

,y,x
3057,ham,"[webpage, available]"
4211,ham,"[dahe, stupid, daalways, sending, like, thisdo..."
1581,ham,"[shall, book, chez, jules, half, eight, thats,..."
1364,ham,"[yetunde, im, sorry, moji, seem, busy, able, g..."
657,ham,"[sun, cant, come, earth, send, luv, ray, cloud..."
...,...,...
1704,ham,"[taste, fish, curry, p]"
2998,ham,"[knowhe, watching, film, computer]"
2128,ham,"[tessypls, favor, pls, convey, birthday, wish,..."
5240,ham,"[gud, gudk, chikku, tke, care, sleep, well, gu..."


In [19]:
def holdout_validationRatio7_3(df):
    train = df.sample(frac=0.8, random_state=200)
    test = df.drop(train.index)
    return train, test

In [20]:
train, test= holdout_validationRatio7_3(sf)
tr = train.reset_index(drop=True)
te = test.reset_index(drop=True)

NAIVES BAYES

Frequences of each word

In [21]:
def calculFrequence(df_):
    df_ = df_.reset_index(drop=True)
    freq_ ={}
    cpt =0
    for i in df_['x']:
        c = Counter(i)
        for key in c:
            if key in freq_.keys(): 
                if df_['y'][cpt] == "ham":
                    freq_[key][1] += c[key]
                else:
                    freq_[key][0] += c[key]    
            else:
                freq_[key]= list()
                if df_['y'][cpt] == "ham":
                    freq_[key].append(0) #Fist element is the number of spam
                    freq_[key].append(c[key]) #Second is the number of ham
                else:
                    freq_[key].append(c[key])
                    freq_[key].append(0)
        cpt+=1
    return freq_

In [22]:
freq = calculFrequence(tr)

Probability to be a spam

In [23]:
Pspam = tr['y'].value_counts()['spam'] / tr.shape[0]
Pham = tr['y'].value_counts()['ham'] / tr.shape[0]

In [24]:
NumberTotalWords = [sum(value) for value in zip(*freq.values())]

NbWordsSpam = NumberTotalWords[0]

NbWordsHam = NumberTotalWords[1]

In [25]:
#LAPLACE SMOOTHING
V = 1*len(freq) #Number of total distinct words
def p_w_spam(word, freq):
    if word in freq.keys():
        return (freq[word][0] +1)/(NbWordsSpam + V)
    else:
        return 1

def p_w_ham(word, freq):
    if word in freq.keys():
        return (freq[word][1] +1)/(NbWordsHam + V)
    else:
        return 1

In [26]:
def classify(message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_w_spam(word, freq)
        p_ham_given_message *= p_w_ham(word, freq)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

PREDICTIONS

In [27]:
te['predicted'] = te['x'].apply(classify)

In [29]:
def print_info(TP, FP, FN, TN, te):
    """
    This function print infomations about our predictions
    """
    accuracy = (te['predicted'] == te['y']).sum() / te.shape[0] * 100
    recall = TP / (TP + FN) *100
    precision = TP / (TP + FP) *100
    print("\nConfusion Matrix")
    print("\t\t    Predict cluster1 \t Predict cluster2")
    print("Predict cluster1 | ", TP,"\t\t\t", FP)
    print("Predict cluster2 | ", FN,"\t\t\t", TN)
    print("\n")
    print("Accuracy: ", accuracy)
    print("Precision (Successfully predict cluster1): ", precision)
    print("Recall (Successfully predict cluster2): ", recall)
    print("F1 score : ", 2*(recall * precision)/(recall + precision))


In [30]:
"""
    CONFUSION MATRIX
"""
def confusion(prediction, true_values,te):
    """
    This fuctions calculte informations of the confusion matrix and 
    give a good view of the predictions using probabilities to convert
    in integer
    Input:
        prediction: list of predictions
        B: matrix of our real values 100 1
    Output:
        TP: integer reresenting True positive 
        FP: integer reresenting False positive
        FN: integer reresenting False negative
        TN: integer reresenting True negatice
        pred: intger list of preidictions 0 or 1
    """
    TP, FP, FN, TN = 0,0,0,0

    for i in range(len(true_values)):
        if true_values[i] == "ham" and prediction[i]== "ham":
            TN += 1
        if true_values[i] == "spam" and prediction[i]== "ham":
            FN += 1
        if true_values[i] == "ham" and prediction[i]== "spam":
            FP += 1
        if true_values[i] == "spam" and prediction[i]== "spam":
            TP += 1
    print_info(TP, FP, FN, TN, te)

In [31]:
confusion(te['predicted'], te['y'],te)


Confusion Matrix
		    Predict cluster1 	 Predict cluster2
Predict cluster1 |  137 			 6
Predict cluster2 |  8 			 963


Accuracy:  98.74326750448833
Precision (Successfully predict cluster1):  95.8041958041958
Recall (Successfully predict cluster2):  94.48275862068965
F1 score :  95.1388888888889
